# CSV file generation
CSV file with image_name and labes exist. The goal is to read the image_name out of the CSV file open it and generate all image features and saving them to the CSV file again. 

In [48]:
import math
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np

### 1. necessary functions

#### 1.0 Loading function

In [49]:
from PIL import Image
import os
import numpy as np

def load_data(load_file_path):
    # Funktion zum Laden von Daten aus dem Ordner load_file_path als TIFF
    # Argument: load_file_path als String
    image_array = []  # Leeres Array zum Speichern der Bilder

    # Iteriere über die Dateien im Ordner
    for file_name in os.listdir(load_file_path):
        file_path = os.path.join(load_file_path, file_name)

        # Überprüfe, ob die Datei ein unterstütztes Bildformat hat
        if file_name.endswith(('.png', '.PNG', '.jpg', '.JPG', '.jpeg', '.JPEG', '.tiff', '.TIFF', '.tif', '.TIF')):
            # Lade das Bild mit PIL
            pil_image = Image.open(file_path)

            # Überprüfe den Datentyp des Bildes
            if isinstance(pil_image, np.ndarray):
                # Wenn das Bild bereits ein NumPy-Array ist, füge es direkt zum Array hinzu
                image_array.append(pil_image)
            else:
                # Wenn das Bild ein PIL Image-Objekt ist, konvertiere es in ein NumPy-Array und füge es hinzu
                image_array.append(np.array(pil_image))

    return image_array

#### 1.1 Color balancing function

In [50]:
import cv2

def apply_color_balance(image_bgr):
    # Konvertierung des Bildes in den LAB-Farbraum
    adjusted_image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2LAB)

    # Aufteilen des Bildes in die einzelnen Kanäle
    b_channel, g_channel, r_channel = cv2.split(adjusted_image)

    # Erstellen eines adaptiven Histogrammausgleichsobjekts mit den angegebenen Parametern
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    # Anwendung des Histogrammausgleichs auf den r-Kanal
    clahe_b_channel = clahe.apply(b_channel)

    # Zusammenführen der Kanäle zu einem Bild
    adjusted_image = cv2.merge((clahe_b_channel, g_channel, r_channel))

    # Konvertierung des Bildes zurück zum BGR-Farbraum
    adjusted_image_bgr = cv2.cvtColor(adjusted_image, cv2.COLOR_LAB2BGR)

    return adjusted_image_bgr

#### 1.2 Canny contour detection

In [51]:
import cv2

def image_contours_detection(image_bgr, gaus_blur_ksize, gaus_blur_sigmaX, canny_threshold1, canny_threshold2, canny_aperture_size, dilate_kernel, dilate_iterations, find_cont_mode, find_cont_method):
    # Bild von BRG zu RGB Format
    image_rgb_org = cv2.cvtColor(np.array(image_bgr), cv2.COLOR_BGR2RGB)
    
    # Bild wird von BGR zu gray-scale gewandelt
    gray_image = cv2.cvtColor(np.array(image_bgr), cv2.COLOR_BGR2GRAY)
    
    # Bild wird mit der openCv Funktion "gaussianBlur" unschärfer gemacht
    blur_image = cv2.GaussianBlur(gray_image, gaus_blur_ksize, gaus_blur_sigmaX)
    # Parameter: ksize=(11,11); ksize:Die Größe des Gauß-Filters als Tupel (Breite, Höhe). Größere Werte führen zu einer stärkeren Glättung des Bildes.
    #            sigmaX=0; sigmaX:Die Standardabweichung des Gauß-Filters. Größere Werte führen zu einer stärkeren Glättung des Bildes.

    # Diese Funktion führt die Kantenextraktion mit dem Canny-Algorithmus durch
    canny_image = cv2.Canny(blur_image, canny_threshold1, canny_threshold2, canny_aperture_size)
    # Parameter: threshold1=30; threshold1: Der erste Schwellenwert für die Hysterese. Kanten mit Gradientenwerten größer als threshold2 werden als starke Kanten betrachtet.
    #            threshold2=150; threshold2: Der zweite Schwellenwert für die Hysterese. Kanten mit Gradientenwerten kleiner als threshold1 werden als schwache Kanten betrachtet.
    #            apertureSize=3; apertureSize: Die Größe des Sobel-Kernels zur Berechnung der Gradienten. Der Standardwert ist 3.   


    # Diese Funktion führt eine Dilatation (Ausdehnung) auf dem Bild durch
    dilated_image = cv2.dilate(canny_image, dilate_kernel, iterations=dilate_iterations)
    # Parameter: kernel=(1,1); kernel: Die Strukturierungsmatrix (Kernel) für die Dilatation. Es kann verwendet werden, um die Form der ausgedehnten Objekte zu kontrollieren.
    #            iterations=0; iterations: Die Anzahl der Iterations für die Dilatation. Je größer der Wert, desto stärker wird das Bild ausgedehnt.


    # Diese Funktion findet Konturen im binären Bild
    (cnt, hierarchy) = cv2.findContours(dilated_image.copy(), find_cont_mode, find_cont_method)
    # Parameter: mode=cv2.RETR_EXTERNAL; Die Kontur-Rückverfolgungsmodi. Häufig verwendete Modi sind cv2.RETR_EXTERNAL (nur die äußeren Konturen) und cv2.RETR_TREE (alle Konturen mit Hierarchie)
    #            method=cv2.CHAIN_APPROX_NONE; method: Die Approximationsmethode für die Kontur. Häufig verwendete Methoden sind cv2.CHAIN_APPROX_NONE (keine Approximation, alle Punkte der Kontur
    #                                                  werden beibehalten) und cv2.CHAIN_APPROX_SIMPLE (nur die Endpunkte der Kontur werden beibehalten).


    # Diese Funktion wandelt das Bild von BGR zu RGB-Format
    image_rgb_contours = cv2.cvtColor(np.array(image_bgr), cv2.COLOR_BGR2RGB)

    # Diese Funktion zeichnet die Konturen auf dem Bild in RGB-Format (hier: grün)
    cv2.drawContours(image_rgb_contours, cnt, -1, (0, 255, 0), 1)

    array_images=[image_rgb_org, gray_image, blur_image, canny_image, dilated_image, image_rgb_contours]
    array_titles=['image_rgb_org', 'gray_image', 'blur_image', 'canny_image', 'dilated_image', 'image_rgb_contours_detect: ' + str(len(cnt))]

    # Rückgabe aller Bilder mit Titeln und der Anzahl der Konturen
    return array_images, array_titles, cnt

#### 1.3 function for creating a mask

In [52]:
import numpy as np
import cv2

def mask_image_outside_contour(image, contours):
    array_layers =[]
    for contour in contours:

        # Erstelle eine Maske für den Bereich außerhalb der Kontur
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [contour], 0, 255, -1)  # Zeichne die Kontur auf die Maske

        # Setze alle Pixel außerhalb der Kontur auf Schwarz
        layer = cv2.bitwise_and(image, image, mask=mask)
        array_layers.append(layer)
    output_image = np.zeros_like(image)
    for layer in array_layers:
        output_image = output_image + layer
    return output_image

#### 1.4 Color segmentation HSV colorspace

In [53]:
import numpy as np
import cv2

def segment_images_hsv(image_bgr, cr_green, cr_red_1, cr_red_2, cr_yellow, cr_blue, cr_white, cr_brown, cr_purple): # color ranges in HSV FORMAT
    # convert the image from BGR to HSV color space
    image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)

    # define the list of boundaries (lower and upper color bounds) in HSV format
    boundaries = [cr_green, cr_red_1, cr_red_2 , cr_yellow, cr_blue, cr_white, cr_brown, cr_purple]

    array_image_bgr_color_segmented = []

    # loop over the boundaries
    for (lower, upper) in boundaries:
        # create NumPy arrays from the boundaries for lower and upper bounds
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")

        # find the colors within the specified boundaries and apply the mask
        mask = cv2.inRange(image_hsv, lower, upper)
        output_bgr = cv2.bitwise_and(image_bgr, image_bgr, mask=mask)
        array_image_bgr_color_segmented.append(output_bgr)

            # green = 0; red_1 = 1; red_2 = 2; yellow = 3; blue = 4; white = 5; borwn = 6; purple = 7; 

    red_combined = np.clip(array_image_bgr_color_segmented[1]+ array_image_bgr_color_segmented[2], 0,255)
    array_image_bgr_color_segmented.append(red_combined)
    blue_combined = cv2.subtract(array_image_bgr_color_segmented[4],array_image_bgr_color_segmented[7]).astype(np.uint8)
    array_image_bgr_color_segmented.append(blue_combined)
    substract_image = array_image_bgr_color_segmented[0] + array_image_bgr_color_segmented[3] + array_image_bgr_color_segmented[4]+ array_image_bgr_color_segmented[5]+ array_image_bgr_color_segmented[7] + + array_image_bgr_color_segmented[8]
    brown_combined = cv2.subtract(array_image_bgr_color_segmented[6], substract_image).astype(np.uint8)
    array_image_bgr_color_segmented.append(brown_combined)

    return array_image_bgr_color_segmented

In [54]:
import csv

# Pfad zur CSV-Datei
file_path = r"C:\Users\SeppG\VdKI\00_project_abgabe\04_CSV\All_Labels.csv"

# CSV-Datei öffnen und erste Zeile auslesen
with open(file_path, 'r') as file:
    reader = csv.reader(file)
    first_row = next(reader)
    print(first_row)

    num_rows = sum(1 for row in reader)
    print("Die CSV-Datei hat", num_rows, "Zeilen.")   

['filename', 'label', 'con_num', 'con_area', 'orb_num_front', 'orb_num_back', 'cs_green', 'cs_yellow', 'cs_white', 'cs_purple', 'cs_red', 'cs_blue', 'cs_brown', 'mod_length', 'mod_length_wo_white', 'mod_area', 'mod_area_wo_white']
Die CSV-Datei hat 2736 Zeilen.


In [55]:
load_file_path_orb = r"C:\Users\SeppG\VdKI\00_project_abgabe\04_CSV\ORB_images"
array_image_rgb_ORB = load_data(load_file_path_orb)

for i in range(num_rows-1): #num_rows
    array_features = []
    # Position des Elements, auf das zugegriffen werden soll
    row_index = i+1  # Index der gewünschten Zeile (0-basiert)
    column_index = 0  # Index der gewünschten Spalte (0-basiert)

    # CSV-Datei öffnen und auf Element zugreifen
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        
        # Zum gewünschten Element navigieren
        for _ in range(row_index):
            next(reader)  # Zeilen überspringen
            
        row = next(reader)  # Gewünschte Zeile lesen
        file_name = row[column_index]  # Gewünschtes Element der Zeile

    print("Image ", file_name, " is processed. ROW:", i+1, )

    import cv2
    folder_file_path = r"C:\Users\SeppG\VdKI\00_project_abgabe\04_CSV\Bilder"
    load_file_path = os.path.join(folder_file_path, file_name)

    test_image_bgr = cv2.imread(load_file_path)
    test_image_rgb = cv2.cvtColor(test_image_bgr, cv2.COLOR_BGR2RGB)

    #color adjustment
    image_bgr_adjusted = apply_color_balance(test_image_bgr)
    image_rgb_adjusted = cv2.cvtColor(image_bgr_adjusted, cv2.COLOR_BGR2RGB)

    gaus_blur_ksize = (13, 13)
    gaus_blur_sigmaX = 1.5 #0.1
    canny_threshold1 = 30
    canny_threshold2 = 120
    canny_aperture_size = 3
    dilate_kernel = (1, 1)
    dilate_iterations = 0
    find_cont_mode =cv2.RETR_TREE
    #find_cont_mode = cv2.RETR_EXTERNAL
    find_cont_method = cv2.CHAIN_APPROX_NONE

    array_image_canny_f_2_1, array_titles_canny_f_2_1, contour_canny_f_2_1 = image_contours_detection(image_bgr_adjusted, gaus_blur_ksize, gaus_blur_sigmaX, canny_threshold1, canny_threshold2, canny_aperture_size, dilate_kernel, dilate_iterations, find_cont_mode, find_cont_method)

    con_num = len(contour_canny_f_2_1) # image feature 1 ***************************************************************************
    array_features.append(con_num)

    total_area = 0
    # Iterate over the contours and add the area of each contour to the total area
    for contour in contour_canny_f_2_1:
        area = cv2.contourArea(contour)
        total_area += area

    # Print the total area
    con_area = total_area # image feature 2 *****************************************************************************************
    array_features.append(con_area)

    array_image_bgr_ORB =[]
    for image_rgb in array_image_rgb_ORB:
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        array_image_bgr_ORB.append(image_bgr)

    image_gray_query = cv2.cvtColor(image_bgr_adjusted, cv2.COLOR_BGR2GRAY)

    keypoints_matches_front_number = 0
    keypoints_matches_back_number = 0
    k=0
    for image_bgr_train in array_image_bgr_ORB:
        # adjust BGR to Gray Colorformat
        image_gray_train = cv2.cvtColor(image_bgr_train, cv2.COLOR_RGB2GRAY)

        # Initiate ORB detector
        orb = cv2.ORB_create()
        # find the keypoints and descriptors with ORB
        keypoints_query, descriptors_query = orb.detectAndCompute(image_gray_query,None)
        keypoints_train, descriptors_train = orb.detectAndCompute(image_gray_train,None)

        if len(keypoints_query) > 0 and len(keypoints_train) > 0:
            # create BFMatcher object
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            # Match descriptors.
            matches = bf.match(descriptors_query,descriptors_train)

            k+=1
            if k % 2 ==1:
                keypoints_matches_front_number = keypoints_matches_front_number + len(matches)
            else:
                keypoints_matches_back_number = keypoints_matches_back_number + len(matches)

            # Sort them in the order of their distance.
            matches = sorted(matches, key = lambda x:x.distance)
            # Draw  matches.
            image_gray_matched_features = cv2.drawMatches(image_gray_query, keypoints_query, image_gray_train, keypoints_train, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

            # Adjusting colorspace
            image_rgb_query = cv2.cvtColor(image_bgr_adjusted, cv2.COLOR_BGR2RGB)
            image_rgb_train = cv2.cvtColor(image_bgr_train, cv2.COLOR_BGR2RGB)

    orb_num_front = keypoints_matches_front_number # image feature 3 **************************************************************
    array_features.append(orb_num_front)
    orb_num_back =  keypoints_matches_back_number # image feature 4 ***************************************************************
    array_features.append(orb_num_back)

    # creating a semi good cropped image
    gaus_blur_ksize = (55, 55)
    gaus_blur_sigmaX = 0.7 #0.1
    canny_threshold1 = 50
    canny_threshold2 = 250
    canny_aperture_size = 3
    dilate_kernel = (33, 33)
    dilate_iterations = 0
    find_cont_mode = cv2.RETR_EXTERNAL
    find_cont_method = cv2.CHAIN_APPROX_NONE

    array_image_canny_f_2_4, array_titles_canny_f_2_4, contour_canny_f_testing_2_4 = image_contours_detection(image_bgr_adjusted, gaus_blur_ksize, gaus_blur_sigmaX, canny_threshold1, canny_threshold2, canny_aperture_size, dilate_kernel, dilate_iterations, find_cont_mode, find_cont_method)

    # apply contour as a mask 
    cut_test_image_bgr_2_4 = mask_image_outside_contour(image_bgr_adjusted, contour_canny_f_testing_2_4)
    cut_test_image_rgb_2_4 = cv2.cvtColor(cut_test_image_bgr_2_4, cv2.COLOR_BGR2RGB)

    # ceating a good cropped image
    gaus_blur_ksize = (55, 55)
    gaus_blur_sigmaX = 0.2 #0.1
    canny_threshold1 = 50
    canny_threshold2 = 250
    canny_aperture_size = 3
    dilate_kernel = (33, 33)
    dilate_iterations = 5
    find_cont_mode = cv2.RETR_EXTERNAL
    find_cont_method = cv2.CHAIN_APPROX_NONE

    array_image_canny_f_2_4, array_titles_canny_f_2_4, contour_canny_f_testing_2_4 = image_contours_detection(cut_test_image_bgr_2_4, gaus_blur_ksize, gaus_blur_sigmaX, canny_threshold1, canny_threshold2, canny_aperture_size, dilate_kernel, dilate_iterations, find_cont_mode, find_cont_method)
    
    # apply contour as a mask 
    cut_test_image_bgr_2_41 = mask_image_outside_contour(image_bgr_adjusted, contour_canny_f_testing_2_4)
    cut_test_image_rgb_2_41 = cv2.cvtColor(cut_test_image_bgr_2_41, cv2.COLOR_BGR2RGB)

    # COLOR SEGMENTATION
    # color rang preset for each color 
    #           HSV FORMATTYPE: HSV ranges between (0-180, 0-255, 0-255)
    #        ([hue_lower, saturation_lower, value_lower],[hue_upper, saturation_upper, value_upper])

    cr_green_2 = ([45,50,5], [75,255,254])

    cr_red_2_1 = ([0,147,111], [6,255,255])
    cr_red_2_2 = ([154,166,66], [180,255,255])

    cr_yellow_2 = ([14,25,0], [25,255,254])

    cr_blue_2 = ([85,70,25], [125,255,255]) 

    cr_white_2 = ([0,0,200], [180,30,255])

    cr_brown_2 = ([0,0,0],[180,197,103])

    cr_purple_2 = ([103,0,0],[168,215,69])

    array_image_bgr_from_hsv_segmented_2 = segment_images_hsv(cut_test_image_bgr_2_41, cr_green_2, cr_red_2_1, cr_red_2_2, cr_yellow_2, cr_blue_2, cr_white_2, cr_brown_2, cr_purple_2)


    array_image_rgb_from_hsv_segmented_2 = []
    for image_bgr in array_image_bgr_from_hsv_segmented_2:
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        array_image_rgb_from_hsv_segmented_2.append(image_rgb)
    
    array_image_rgb_selected_2_4 = [array_image_rgb_from_hsv_segmented_2[0],array_image_rgb_from_hsv_segmented_2[3],array_image_rgb_from_hsv_segmented_2[5],
                                array_image_rgb_from_hsv_segmented_2[7],array_image_rgb_from_hsv_segmented_2[8],array_image_rgb_from_hsv_segmented_2[9],array_image_rgb_from_hsv_segmented_2[10]]
    array_image_bgr_selected_2_4 = []
    for image_rgb in array_image_rgb_selected_2_4:
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        array_image_bgr_selected_2_4.append(image_bgr)
    
    # Median Blur: This technique replaces each pixel value with the median value in its neighborhood, which is effective in removing salt-and-pepper noise.
    # Bilateral Filter: This technique preserves the edges in the image while reducing noise by considering both spatial and intensity differences.

    array_image_bgr_medianblur = []
    for image_bgr in array_image_bgr_selected_2_4:
        image_blurred = cv2.medianBlur(image_bgr, 7)
        array_image_bgr_medianblur.append(image_blurred)

    array_image_bgr_bilateral = []
    for image_bgr in array_image_bgr_medianblur:
        image_bgr = cv2.bilateralFilter(image_bgr, 19 , 75, 75)
        array_image_bgr_bilateral.append(image_bgr)

    array_image_rgb_bilateral =[]
    for image_bgr in array_image_bgr_bilateral:
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        array_image_rgb_bilateral.append(image_rgb)
    
    # transformation to binary color space for higher contrast
    threshold_value = 5
    max_value = 160

    array_image_gray_binary =[]
    for image_bgr in array_image_bgr_bilateral:
        # Konvertiere das Bild in den richtigen Farbraum (Graustufen)
        image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)

        # Wende eine Schwellenwertoperation an, um das Bild zu binarisieren
        _, binary_image = cv2.threshold(image_gray, threshold_value, max_value, cv2.THRESH_BINARY)
        array_image_gray_binary.append(binary_image)
    
    array_contour_numbers_binary = []
    array_image_rgb_color_segmented= []
    total_arc_length_with_white = 0
    total_arc_length_without_white = 0
    total_area_with_white = 0
    total_area_without_white = 0

    k=0

    for image_gray_binary in array_image_gray_binary:
        # Führe eine Konturfindung auf dem binarisierten Bild durch
        contours, _ = cv2.findContours(image_gray_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) > 0:
            # Berechne die größte Fläche einer Kontur
            largest_area = max([cv2.contourArea(contour) for contour in contours])

            # Iteriere über alle gefundenen Konturen
            filtered_contours = []
            for contour in contours:
                # Überprüfe, ob die Fläche der Kontur mindestens die Hälfte der größten Fläche beträgt
                if cv2.contourArea(contour) >= largest_area / 8:
                    filtered_contours.append(contour)
            
            area_sum = 0
            arc_length_sum = 0

            for contour in filtered_contours:
                area = cv2.contourArea(contour)
                arc_length = cv2.arcLength(contour, True)
                area_sum += area
                arc_length_sum += arc_length
            
            if k == 2: # wenn 4 wird das weiß segementierte Bild berechnet
                total_arc_length_with_white += arc_length_sum
                total_area_with_white = area_sum
            else: 
                total_arc_length_with_white += arc_length_sum
                total_arc_length_without_white += arc_length_sum

                total_area_with_white += area_sum
                total_area_without_white += area_sum
        

            # Zähle die identifizierten Objekte
            count = len(filtered_contours)
            if k  == 0:
                cs_green = count # image feature 5 *******************************************
                array_features.append(cs_green)
            elif k == 1:
                cs_yellow = count # image feature 6 *******************************************
                array_features.append(cs_yellow)
            elif k == 2:
                cs_white = count  # image feature 7 *******************************************
                array_features.append(cs_white)
            elif k == 3:
                cs_purple = count # image feature 8 *******************************************
                array_features.append(cs_purple)
            elif k == 4:
                cs_red = count    # image feature 9 *******************************************
                array_features.append(cs_red)
            elif k == 5:
                cs_blue = count   # image feature 10 *******************************************
                array_features.append(cs_blue)
            elif k == 6:
                cs_brown = count  # image feature 11 *******************************************
                array_features.append(cs_brown)
        else:
            if k  == 0:
                cs_green = 0 # image feature 5 *******************************************
                array_features.append(cs_green)
            elif k == 1:
                cs_yellow = 0 # image feature 6 *******************************************
                array_features.append(cs_yellow)
            elif k == 2:
                cs_white = 0  # image feature 7 *******************************************
                array_features.append(cs_white)
            elif k == 3:
                cs_purple = 0 # image feature 8 *******************************************
                array_features.append(cs_purple)
            elif k == 4:
                cs_red = 0    # image feature 9 *******************************************
                array_features.append(cs_red)
            elif k == 5:
                cs_blue = 0   # image feature 10 *******************************************
                array_features.append(cs_blue)
            elif k == 6:
                cs_brown = 0  # image feature 11 *******************************************
                array_features.append(cs_brown)        
        k += 1

    mod_length = total_arc_length_with_white             # image feature 12 *******************************************
    array_features.append(mod_length)
    mod_length_wo_white = total_arc_length_without_white # image feature 13 *******************************************
    array_features.append(mod_length_wo_white)
    mod_area =  total_area_with_white                    # image feature 14 *******************************************
    array_features.append(mod_area)
    mode_area_wo_white = total_area_without_white        # image feature 15 *******************************************
    array_features.append(mode_area_wo_white)
    if len(array_features) <15:
        print("There were:", len(array_features))
        print(array_features)
    
    # Position der Zeile, in die die Werte geschrieben werden sollen
    row_index = i+1  # Index der gewünschten Zeile (0-basiert)

    # Spaltenindex, ab dem die Werte eingefügt werden sollen
    start_column_index = 2  # Index der ersten Spalte (0-basiert)

    # CSV-Datei öffnen und Daten schreiben
    with open(file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        rows = list(reader)  # Alle Zeilen der CSV-Datei lesen

    # Überprüfen, ob die ausgewählte Zeile existiert
    if row_index < len(rows):
        # Anzahl der vorhandenen Spalten in der ausgewählten Zeile
        num_existing_columns = len(rows[row_index])

        # Überprüfen, ob genügend Spalten vorhanden sind
        if start_column_index + len(array_features) <= num_existing_columns:
            # Werte in die ausgewählte Zeile schreiben
            for i, value in enumerate(array_features):
                column_index = start_column_index + i
                rows[row_index][column_index] = value
        else:
            # Anzahl der fehlenden Spalten
            num_missing_columns = start_column_index + len(array_features) - num_existing_columns

            # Neue Spalten hinzufügen
            for _ in range(num_missing_columns):
                rows[row_index].append("")

            # Werte in die ausgewählte Zeile schreiben
            for i, value in enumerate(array_features):
                column_index = start_column_index + i
                rows[row_index][column_index] = value
    else:
        print("Die ausgewählte Zeile existiert nicht.")

    # Aktualisierte Daten in die CSV-Datei schreiben
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

        

Image  IMG_4064.png  is processed. ROW: 1
Image  IMG_4070.png  is processed. ROW: 2
Image  IMG_4058.png  is processed. ROW: 3
Image  IMG_3723.png  is processed. ROW: 4
Image  IMG_3910.png  is processed. ROW: 5
Image  IMG_3904.png  is processed. ROW: 6
Image  IMG_3938.png  is processed. ROW: 7
Image  IMG_4110.png  is processed. ROW: 8
Image  IMG_4104.png  is processed. ROW: 9
Image  IMG_3643.png  is processed. ROW: 10
Image  IMG_3657.png  is processed. ROW: 11
Image  IMG_4138.png  is processed. ROW: 12
Image  IMG_3864.png  is processed. ROW: 13
Image  IMG_3870.png  is processed. ROW: 14
Image  IMG_3858.png  is processed. ROW: 15
Image  IMG_3680.png  is processed. ROW: 16
Image  IMG_3694.png  is processed. ROW: 17
Image  IMG_3695.png  is processed. ROW: 18
Image  IMG_3681.png  is processed. ROW: 19
Image  IMG_3859.png  is processed. ROW: 20
Image  IMG_3871.png  is processed. ROW: 21
Image  IMG_3865.png  is processed. ROW: 22
Image  IMG_4139.png  is processed. ROW: 23
Image  IMG_3656.png 